In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

/workspace


In [2]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

DEBUG:__main__:hello
DEBUG:lib.lab_lib:hello
